### City similarity trials

##### With aim to find the similarity between cities, we're trying to match the attributes that defines a city. In our case few attributes can be mountain, beach, spiritual etc given in dataset.
##### Here the Ideas is to finding nearest neighbours. To achieve this we vae to calculate the distance between data points

## Data preparation

In [4]:
import pandas as pd

df = pd.read_csv('City_data2.csv')
df1 = df[['city','beach','mountain','history','food','city_life','countryside','nightlife','couple_friendly','outdoor','spiritual','hot','cold','mild','snowy','desert','rainy','athletics']]
df1.head()

,city,beach,mountain,history,food,city_life,countryside,nightlife,couple_friendly,outdoor,spiritual,hot,cold,mild,snowy,desert,rainy,athletics
0,Paris,0,0,1,1,1,0,1,1,0,0,0,1,1,1,0.0,1,1
1,New york,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0.0,1,1
2,Rome,1,0,1,1,1,1,1,1,1,1,1,0,1,0,0.0,0,0
3,London,1,1,1,1,1,0,1,1,0,0,0,1,1,1,0.0,1,1
4,Tokyo,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0.0,0,1


In [6]:
df2 = df1.set_index('city').T.to_dict('list')
df2

{'Paris': [0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0],
 'New york': [0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0],
 'Rome': [1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'London': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0],
 'Tokyo': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0],
 'Lisbon': [1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Barcelona': [1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0],
 'Honolulu': [1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,


In [7]:
import numpy as np
data = dict()
names = []
vectors = []


for a,b in df2.items():
    names.append(a)
    vectors.append(b)

data['name'] = np.array(names, dtype=object)
data['vector'] = np.array(vectors, dtype=float)

### V0.1 ANN - Approximate nearest neighbour
##### https://github.com/spotify/annoy

###### Here we will define dimentions and vectors for search critaria. ANNOY will take care of it.

In [11]:
# !pip install annoy

In [15]:
import annoy

class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [16]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

<ipython-input-15-7a85ecbad9b0>:11: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  self.index = annoy.AnnoyIndex(self.dimention)


In [24]:
#Let's try now
import time
place_vector, place_name = data['vector'][96], data['name'][96]
start = time.time()
simlar_place_names = '\n* '.join(index.query(place_vector))
print(simlar_place_names)
end = time.time()
print('time taken '+ str(end - start))

Taipei
* Caracas
* Kyoto
* San Salvador
* Arequipa
* Brisbane
* Crete 
* Bath
* Wellington
* Nice
time taken 0.0035538673400878906


## Is it the best we can?

### Let's try something we learnt, May be LSH so that we can eliminate some false positives

## v0.2 LSH

## Validation
#### Result validation will be done using 'Dynamic time warping'.
#### Its a technique normally used in speech domain where we can consider 2 dimentions such as frequency and time parameter. I'm going to use it as a scale metrics to see the distance between two vectors.

In [53]:
# check the city_sim val file
